In [49]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [50]:
# Load datas
train_data=pd.read_csv('train.csv')
test_data=pd.read_csv('test.csv')

train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [51]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [52]:
train_data.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


**Missing values**

We gonna check how many missing values we have for each variable.

In [53]:
train_data.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [54]:
# we delete the 2 rows with missings values in 'Embarked'
train_data.dropna(subset=['Embarked'], inplace=True)

In [55]:
train_data.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         0
dtype: int64

In [56]:
test_data.isna().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [57]:
# for 'Age' we are going to implement missing values by using the mean() method because we don't have outliers here
train_data['Age'].fillna(train_data['Age'].median(), inplace=True)

test_data['Age'].fillna(test_data['Age'].median(), inplace=True)

# for fare in test_data we're going to do the same
test_data['Fare'].fillna(test_data['Fare'].median(), inplace=True)

In [58]:
train_data=train_data.drop(columns=['Name', 'Ticket'])

test_data=test_data.drop(columns=['Name','Ticket'])

In [59]:
# we keep only the first letter of the cabin correspondick to the deck
train_data['Deck']=train_data['Cabin'].str[0]

# we give 'U' to unknown cabins
train_data['Deck'].fillna('U', inplace=True)

# then we delete 'cabin'
train_data=train_data.drop(columns='Cabin')

# then we do the same for test_data
test_data['Deck']=test_data['Cabin'].str[0]
test_data['Deck'].fillna('U', inplace=True)
test_data=test_data.drop(columns='Cabin')

**Transform categorical variables**

In [60]:
# we encode 'sex' with 0 and 1
train_data.loc[train_data.Sex=='male','Sex']=0
train_data.loc[train_data.Sex=='female', 'Sex']=1

test_data.loc[test_data.Sex=='male','Sex']=0
test_data.loc[test_data.Sex=='female', 'Sex']=1

In [61]:
train_data['Embarked'].value_counts()

Embarked
S    644
C    168
Q     77
Name: count, dtype: int64

In [62]:
test_data['Embarked'].value_counts()

Embarked
S    270
C    102
Q     46
Name: count, dtype: int64

For 'Embarked' we gonna use one-hot encoding because we have only 3 values for this variable so we are not going to create to many variables (only 2 !)

In [63]:
# we use get_dummies() function from pandas, we take of one column because it's implicite
train_data=pd.get_dummies(train_data, columns=['Embarked'], drop_first=True)

test_data=pd.get_dummies(test_data, columns=['Embarked'], drop_first=True)

For encoding 'Deck' variable we gonna use LabelEncoder() function because we don't want to create a specific order here :

In [64]:
from sklearn.preprocessing import LabelEncoder

le=LabelEncoder()
train_data['Deck']=le.fit_transform(train_data['Deck'])
test_data['Deck']=le.fit_transform(test_data['Deck'])

In [65]:
from sklearn.model_selection import train_test_split

y=train_data.Survived
X=train_data.drop(columns=['Survived'])

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=808)

In [66]:
from sklearn.linear_model import LogisticRegression
lr_model=LogisticRegression()

lr_model.fit(X_train, y_train)

/usr/lib/python3/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [67]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

y_pred=lr_model.predict(X_test)
print(f'Accuracy : {accuracy_score(y_test, y_pred)}')
print(f'Confusion matrix : {confusion_matrix(y_test, y_pred)}')
print(f'Classification report : {classification_report(y_test,y_pred)}')

Accuracy : 0.7921348314606742
Confusion matrix : [[107  14]
 [ 23  34]]
Classification report :               precision    recall  f1-score   support

           0       0.82      0.88      0.85       121
           1       0.71      0.60      0.65        57

    accuracy                           0.79       178
   macro avg       0.77      0.74      0.75       178
weighted avg       0.79      0.79      0.79       178



The accuracy of this model is 0.79 which is not so bad for a simple model. However we see the model has more difficulties to correctly find survivors. Now we gonna look what happens if we use a DecisionTreeClassifier.

In [68]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score

clf=DecisionTreeClassifier(max_depth=3, random_state=808)
clf.fit(X_train, y_train)


auc=roc_auc_score(y_test,clf.predict_proba(X_test)[:,1], multi_class='ovr')
print('AUC = ', auc)

y_test_hat=clf.predict(X_test)
print(f'Classification report : {classification_report(y_test, y_test_hat)}')

AUC =  0.8173118747281428
Classification report :               precision    recall  f1-score   support

           0       0.84      0.88      0.86       121
           1       0.72      0.63      0.67        57

    accuracy                           0.80       178
   macro avg       0.78      0.76      0.77       178
weighted avg       0.80      0.80      0.80       178



I tried to use GridSearchCV to optimize the max_depth :

In [69]:
from sklearn.model_selection import GridSearchCV
import numpy as np

# we create different values for max_depth from 2 to 30
parameters={'max_depth':np.arange(2,30,2)}

# we gonna compare the auc score of the model with different max_depth values
model=DecisionTreeClassifier(random_state=808)
clf=GridSearchCV(model, parameters, cv=5, scoring='roc_auc_ovr', verbose=1)
clf.fit(X,y)

clf.best_estimator_

Fitting 5 folds for each of 14 candidates, totalling 70 fits


DecisionTreeClassifier(max_depth=2, random_state=808)

The GridSearchCV() function tells us the best max_depth is 2 so I run again the DecisionTreeClassifier with this parameter :

In [70]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score

clf=DecisionTreeClassifier(max_depth=2, random_state=808)
clf.fit(X_train, y_train)


auc=roc_auc_score(y_test,clf.predict_proba(X_test)[:,1], multi_class='ovr')
print('AUC = ', auc)

y_test_hat=clf.predict(X_test)
print(f'Classification report : {classification_report(y_test, y_test_hat)}')

AUC =  0.8177468464549804
Classification report :               precision    recall  f1-score   support

           0       0.83      0.87      0.85       121
           1       0.69      0.61      0.65        57

    accuracy                           0.79       178
   macro avg       0.76      0.74      0.75       178
weighted avg       0.78      0.79      0.78       178



We don't see a huge difference here...

In [71]:
from sklearn.ensemble import GradientBoostingClassifier

gb_clf=GradientBoostingClassifier(n_estimators=60, learning_rate=0.1, max_depth=2, random_state=42)
gb_clf.fit(X_train, y_train)

y_pred=gb_clf.predict(X_test)

print('Accuracy : ', accuracy_score(y_test, y_pred))

print('Classification report : \n', classification_report(y_test, y_pred))

Accuracy :  0.8258426966292135
Classification report : 
               precision    recall  f1-score   support

           0       0.85      0.91      0.88       121
           1       0.77      0.65      0.70        57

    accuracy                           0.83       178
   macro avg       0.81      0.78      0.79       178
weighted avg       0.82      0.83      0.82       178



In [72]:
features_importances=gb_clf.feature_importances_
for feature, importance in zip(X.columns, features_importances):
    print(f'{feature} : {importance}')

PassengerId : 0.019489371838618375
Pclass : 0.1410335896269911
Sex : 0.5718603334637146
Age : 0.09950271726216539
SibSp : 0.04138899549274696
Parch : 0.01048656884895026
Fare : 0.03791967252237279
Deck : 0.05586335276794704
Embarked_Q : 0.0
Embarked_S : 0.022455398176493443


In [73]:
X_test_final=test_data[['PassengerId','Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Deck', 'Embarked_Q', 'Embarked_S']]
y_test_pred=gb_clf.predict(X_test_final)


In [74]:
submission=pd.DataFrame({'PassengerId' : test_data['PassengerId'], 'Survived' : y_test_pred})

In [75]:
submission

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,0
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [76]:
submission.to_csv('submission.csv', index=False)

print('Fichier de soumission généré : submission.csv')

Fichier de soumission généré : submission.csv
